# Примерен агент за резервация на хотел и полет

Това решение ще ви помогне да резервирате самолетни билети и хотел. Сценарият е пътуване от Лондон Хийтроу (LHR) на 20 февруари 2024 г. до Ню Йорк (JFK) с връщане на 27 февруари 2025 г., летейки в икономична класа само с British Airways. Искам престой в хотел Hilton в Ню Йорк, моля, предоставете цени за полета и хотела.


# Инициализиране на услугата Azure AI Agent и получаване на информация за конфигурацията от **.env**

### **.env**

Създайте файл .env

**.env** съдържа низ за връзка към услугата Azure AI Agent, модела, използван от AOAI, както и съответната Google API Search услуга, ENDPOINT и други.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Името на вашия модел за разгръщане в услугата Azure AI Agent"

[**NOTE**] Ще ви е необходим модел с лимит от 100,000 токена на минута и лимит от 600 заявки на минута.

  Можете да получите модел в Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Низ за връзка към вашия проект в услугата Azure AI Agent"

  Можете да получите низа за връзка към проекта в прегледа на проекта в екрана на AI Foundry Portal.

- **SERPAPI_SEARCH_API_KEY** = "Вашият SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Вашият SERPAPI Search Endpoint"

За да получите името на модела за разгръщане и низа за връзка към проекта в услугата Azure AI Agent, трябва да създадете Azure AI Agent Service. Препоръчително е да използвате [този шаблон](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json), за да го създадете директно. (***Забележка:*** Услугата Azure AI Agent в момента е налична само в ограничени региони. Препоръчително е да се консултирате с [този линк](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support), за да зададете региона.)

Агентът трябва да има достъп до SERPAPI. Препоръчително е да се регистрирате чрез [този линк](https://serpapi.com/searches). След регистрация можете да получите уникален API KEY и ENDPOINT.


# Настройка

За да стартирате този notebook, трябва да се уверите, че сте инсталирали необходимите библиотеки, като изпълните `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Вашата версия на Semantic Kernel трябва да бъде поне 1.27.2.


Заредете настройките и ресурсите от вашия .env файл, моля, уверете се, че сте добавили вашите ключове и настройки и сте създали локален .env файл.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Вход в Azure

Сега трябва да влезете в Azure. Отворете терминал и изпълнете следната команда:

```bash
az login
```

Тази команда ще ви подкани да въведете вашите Azure идентификационни данни, което ще позволи на услугата Azure AI Agent да функционира правилно.


# Обяснение:
Това е променлива, която съхранява API ключ за достъп до услуга за SERP (Search Engine Results Page). API ключът е уникален идентификатор, използван за удостоверяване на заявки, свързани с вашия акаунт.

Цел: Целта на този ред е да съхрани API ключа в променлива, за да може да се използва за удостоверяване на заявки към услугата SERP API. API ключът е необходим за достъп до услугата и извършване на търсения.
Как да получите SERP API ключ: За да получите SERP API ключ, следвайте тези общи стъпки на https://serpapi.com (точните стъпки може да варират в зависимост от конкретната услуга SERP API, която използвате):

Изберете услуга за SERP API: Има няколко налични услуги за SERP API, като SerpAPI, Google Custom Search JSON API и други. Изберете тази, която най-добре отговаря на вашите нужди.

Регистрирайте се за акаунт: Отидете на уебсайта на избраната услуга SERP API и се регистрирайте за акаунт. Може да се наложи да предоставите основна информация и да потвърдите имейл адреса си.

Създайте API ключ: След като се регистрирате, влезте в акаунта си и отидете в секцията за API или таблото за управление. Потърсете опция за създаване или генериране на нов API ключ.
Копирайте API ключа във вашия .env файл.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Обяснение:
BASE_URL: Това е променлива, която съхранява основния URL за крайната точка на SERP API. Името на променливата BASE_URL е конвенция, използвана за обозначаване, че този URL е началната точка за изпълнение на API заявки.
'https://serpapi.com/search':

Това е действителният URL низ, присвоен на променливата BASE_URL. Той представлява крайна точка за извършване на заявки за търсене чрез SERP API.

# Цел:
Целта на този ред е да дефинира константа, която съдържа основния URL за SERP API. Този URL ще се използва като начална точка за създаване на API заявки за извършване на операции за търсене.

# Употреба:
Чрез дефиниране на основния URL в променлива, можете лесно да го използвате повторно в кода си, когато трябва да изпълнявате заявки към SERP API. Това прави кода ви по-лесен за поддръжка и намалява риска от грешки, произтичащи от твърдо кодиране на URL на множество места. Текущият пример е https://serpapi.com/search?engine=bing, който използва Bing API за търсене. Различни API могат да бъдат избрани на https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Обяснение:

Тук се намира кодът на вашия плъгин.

Дефиниция на клас: `class BookingPlugin`: Дефинира клас с име BookingPlugin, който съдържа методи за резервация на хотели и полети.

Метод за резервация на хотел:

- `@kernel_function(description="booking hotel")`: Декоратор, който описва функцията като kernel функция за резервация на хотели.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Дефинира метод за резервация на хотели с анотирани параметри и тип на връщане.

Методът създава речник с параметри за заявката за резервация на хотел и изпраща GET заявка към SERP API. Проверява статуса на отговора и връща информация за хотелите, ако заявката е успешна, или None, ако заявката е неуспешна.

Метод за резервация на полет:

- `@kernel_function(description="booking flight")`: Декоратор, който описва функцията като kernel функция за резервация на полети.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Дефинира метод за резервация на полети с анотирани параметри и тип на връщане.

Методът създава речници с параметри за заявките за изходящ и връщащ полет и изпраща GET заявки към SERP API. Проверява статуса на отговора и добавя информацията за полета към резултатния низ, ако заявката е успешна, или отпечатва съобщение за грешка, ако заявката е неуспешна. Методът връща резултатния низ, съдържащ информацията за полетите.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Обяснение:
Импорт на модули: Импортиране на необходимите модули за Azure идентификационни данни, AI агент, съдържание на съобщения в чат, роля на автора и декоратор за функции на ядрото.

Асинхронен контекст мениджър: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Това създава асинхронен контекст мениджър за управление на Azure идентификационни данни и създаване на клиент за AI агент.

Име и инструкции на агента:
- `AGENT_NAME = "BookingAgent"`: Определя името на агента.
- `AGENT_INSTRUCTIONS = """..."""`: Осигурява подробни инструкции за агента как да обработва заявки за резервации.

Създаване на дефиниция на агент: `agent_definition = await client.agents.create_agent(...)`: Създава дефиниция на агент с посочения модел, име и инструкции.

Създаване на AzureAI агент: `agent = AzureAIAgent(...)`: Създава AzureAI агент, използвайки клиента, дефиницията на агента и определения плъгин.

Създаване на нишка: `thread: AzureAIAgentThread | None = None`: Създава нишка за агента. Не е задължително първо да се създаде нишка - ако е предоставена стойност `None`, нова нишка ще бъде създадена при първото извикване и върната като част от отговора.

Потребителски входни данни: `user_inputs = ["..."]`: Определя списък с входни данни от потребителя, които агентът трябва да обработи.

В блока finally, изтрийте нишката и агента, за да освободите ресурси.


# Автентикация

Класът `DefaultAzureCredential` е част от Azure SDK за Python. Той предоставя стандартен начин за автентикация с услугите на Azure. Опитва се да се автентикира чрез множество методи в определен ред, като например променливи на средата, управлявана идентичност и идентификационни данни от Azure CLI.

Асинхронни операции: Модулът aio показва, че класът DefaultAzureCredential поддържа асинхронни операции. Това означава, че можете да го използвате с asyncio за извършване на автентикационни заявки без блокиране.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
